<a href="https://colab.research.google.com/github/seemanthini-n/DeepLearning.AI/blob/master/RNN_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path_to_file='/content/djmh.txt' 

import re
inputtext = open('/content/djmh.txt').read()
cleantext = re.sub('[^a-zA-Z0-9\n\.]', ' ', inputtext)
open('/content/djmh.txt', 'w').write(cleantext)

140891

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 140891 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

    START OF THIS PROJECT GUTENBERG EBOOK THE STRANGE CASE OF DR. JEKYLL AND MR. HYDE    

STORY OF THE DOOR

Mr. Utterson the lawyer was a man of a rugged countenance that was
never lighted by a smile  cold  scanty and embarrassed in discourse 
back


In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

59 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '.' :   2,
  '0' :   3,
  '1' :   4,
  '2' :   5,
  '4' :   6,
  '5' :   7,
  '8' :   8,
  'A' :   9,
  'B' :  10,
  'C' :  11,
  'D' :  12,
  'E' :  13,
  'F' :  14,
  'G' :  15,
  'H' :  16,
  'I' :  17,
  'J' :  18,
  'K' :  19,
  ...
}


In [0]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'    START OF ' ---- characters mapped to int ---- > [ 1  1  1  1 27 28  9 26 28  1 23 14  1]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

 
 
 
 
S


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'    START OF THIS PROJECT GUTENBERG EBOOK THE STRANGE CASE OF DR. JEKYLL AND MR. HYDE    \n\nSTORY OF T'
'HE DOOR\n\nMr. Utterson the lawyer was a man of a rugged countenance that was\nnever lighted by a smile '
' cold  scanty and embarrassed in discourse \nbackward in sentiment  lean  long  dusty  dreary and yet '
'somehow\nlovable. At friendly meetings  and when the wine was to his taste \nsomething eminently human '
'beaconed from his eye  something indeed which\nnever found its way into his talk  but which spoke not '


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '    START OF THIS PROJECT GUTENBERG EBOOK THE STRANGE CASE OF DR. JEKYLL AND MR. HYDE    \n\nSTORY OF '
Target data: '   START OF THIS PROJECT GUTENBERG EBOOK THE STRANGE CASE OF DR. JEKYLL AND MR. HYDE    \n\nSTORY OF T'


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 (' ')
  expected output: 1 (' ')
Step    1
  input: 1 (' ')
  expected output: 1 (' ')
Step    2
  input: 1 (' ')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 27 ('S')
Step    4
  input: 27 ('S')
  expected output: 28 ('T')


In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 59) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           15104     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 59)            60475     
Total params: 4,013,883
Trainable params: 4,013,883
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([32, 29, 18, 21, 20, 29, 49, 37, 54, 54, 21,  9, 27,  6, 50, 26,  2,
       13, 34, 32, 58, 56, 45,  4, 51, 46, 56,  1, 38, 43, 43, 34, 17, 25,
       32, 47,  0, 54, 54, 54,  5, 53,  4, 13, 19,  0, 35, 19, 52, 54, 45,
        1, 52, 12, 29,  2,  1, 31, 49, 26, 29, 37, 26, 45, 56, 20,  4,  3,
       51, 38, 48, 45, 38,  2,  4, 40,  0, 53, 41, 15, 56, 52, 21, 37, 23,
       51,  3, 44, 44, 21, 14, 21, 16,  3, 35,  4, 54, 51, 29, 13])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' unlocked the door and disappeared into\nthe house.\n\nThe lawyer stood awhile when Mr. Hyde had left h'

Next Char Predictions: 
 'YUJMLUqevvMAS4rR.EbYzxm1snx fkkbIQYo\nvvv2u1EK\ncKtvm tDU. WqRUeRmxL10sfpmf.1h\nuiGxtMeOs0llMFMH0c1vsUE'


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 59)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.076025


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=100

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

21/21 [==============================] - 3s 139ms/step - loss: 2.9108
Epoch 3/100
21/21 [==============================] - 3s 140ms/step - loss: 2.5989
Epoch 4/100
21/21 [==============================] - 3s 143ms/step - loss: 2.4343
Epoch 5/100
21/21 [==============================] - 3s 138ms/step - loss: 2.3486
Epoch 6/100
21/21 [==============================] - 3s 143ms/step - loss: 2.2807
Epoch 7/100
21/21 [==============================] - 3s 143ms/step - loss: 2.2043
Epoch 8/100
21/21 [==============================] - 3s 141ms/step - loss: 2.1312
Epoch 9/100
21/21 [==============================] - 3s 150ms/step - loss: 2.0575
Epoch 10/100
21/21 [==============================] - 3s 140ms/step - loss: 1.9853
Epoch 11/100
21/21 [==============================] - 3s 140ms/step - loss: 1.9203
Epoch 12/100
21/21 [==============================] - 3s 141ms/step - loss: 1.8606
Epoch 13/100
21/21 [==============================] - 3s 143ms/step - loss: 1.8078
Epoch 14/100
21/21 [====

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

'./training_checkpoints/ckpt_100'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            15104     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 59)             60475     
Total params: 4,013,883
Trainable params: 4,013,883
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            15104     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
______________________

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"Jekyll"))

Jekyll   s favourite  of a man who could afford to laught the
veitof Hyde.   

   Hm     said Mr. Utterson.    What sort of a way  what and you to be sure  but in the lawyer     I see you have some good reason  Poole  I see
there is something seriously amiss. Try   pread cabitele was so much dives and hands. St sight  hid bear
without efficiency. You will learn from Poole how I have had London. But tonight there was a shudder
in his blood  the face of Hyde sat heavy on his memory  he sat or oue  poor Hyde. How was this to be explained 
I asked myself  and then  wreck of my chair  or continue  with
the most strained and fearstruck ecstasy of listening  to pavement on Jekyll   s own confession  I bring the life of that unhappy Henry Jekyll to be seen by foul   if Jekyll will only
let me.    For once more he ored see was of silver  hand of Edward Hyde. Yes 
I shall sive him and began to whisper      is a tall  fine build of a man 
and tht walk with some degree of steadiness
among temptati